<a href="https://colab.research.google.com/github/AlexandreBourrieau/ML/blob/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/Bitcoin/Bitcoin_Identification_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import commons as cmns

import tensorflow as tf

# Téléchargement des données

In [ ]:
!wget --no-check-certificate --content-disposition "https://github.com/AlexandreBourrieau/ML/blob/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/Bitcoin/Bitcoin_complet.zip?raw=true"
!unzip Bitcoin_complet.zip

In [ ]:
df_data=pd.read_csv('Bitcoin_complet.csv')
df_data.shape

In [ ]:
df_data.head()

# Préparation des jeux de données

In [ ]:
# Déplace la colonne Prix au début
col_prix = df_data.pop('Price')
df_data.insert(1,'Price',col_prix)
df_data.head()

In [ ]:
X_raw = df_data.iloc[:,2:20]
X_raw.head()

In [ ]:
y = df_data.iloc[:,1:2]
y.head()

# Identification des variables par Random Forest

In [ ]:
features_list=[]
indicateurs_techniques=['sma','ema','wma','trix', 'std','skew','rsi','roc']
periode=['3','7','14','30','90']

for indicateur in indicateurs_techniques:
    for i in periode:
      print("%s%s" %(indicateur, i))
      filtre = str(indicateur) + str(i) + "$"

      X = df_data.filter(regex=filtre,axis=1)
      X = SimpleImputer(missing_values=np.nan,strategy='most_frequent').fit_transform(X)
      X = pd.DataFrame(X)
      X.columns = df_data.filter(regex=filtre,axis=1).columns

      rf1 = RandomForestRegressor(random_state=7,n_jobs=-1)
      rfecv = RFECV(rf1,step=0.9,min_features_to_select=1,verbose=1,cv=5,scoring='neg_mean_absolute_error', n_jobs=1)
      rfecv.fit(X,tf.squeeze(np.asarray(y),1))

      if rfecv.n_features_ > 1:
        rf1.fit(X,tf.squeeze(np.asarray(y),1))
        maximp = rf1.feature_importances_.max()
        for x in range(len(rf1.feature_importances_)):
          if maximp==rf1.feature_importances_[x]:
            new_features = X.columns[x]
      else:
        mask = rfecv.get_support()
        new_features = X.columns[mask]
      features_list.append(str(new_features))
      print(filtre+ ': ' + new_features)

In [ ]:
features_list

In [ ]:
l1=[]

for j in range(len(features_list)):
    result1 = re.search("'(.*)'],", features_list[j])
    if result1!=None:
        l1.append(result1.group(1))
        
for i in range(len(features_list)):
    result2 = re.search('.*',features_list[i])
    if len(result2.group(0))<33:
        l1.append(result2.group(0))

In [ ]:
l1.sort()
l1

In [ ]:
df_data_reduit=df_data[l1]
df_data_reduit.head()

Suppresion des valeurs non numériques :

In [ ]:
df_data_reduit = SimpleImputer(missing_values=np.nan,strategy='most_frequent').fit_transform(df_data_reduit)
df_data_reduit = pd.DataFrame(X)
df_data_reduit.columns = df_data_reduit.columns
df_data_reduit

# Identifications des variables réstantes collinéraires 

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

#function for removing features with high vif
def drop_high_vif(X, thresh=100):
    cols = X.columns
    variables = np.arange(X.shape[1])
    dropped=True
    while dropped:
        dropped=False
        c = X[cols[variables]].values
        vif = [variance_inflation_factor(c, ix) for ix in np.arange(c.shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X[cols[variables]].columns[maxloc] + '\' at index: ' + str(maxloc))
            variables = np.delete(variables, maxloc)
            dropped=True

    print('Remaining variables:')
    print(X.columns[variables])
    return X[cols[variables]]

In [ ]:
df_data_reduit = drop_high_vif(df_data_reduit,thresh=5)
df_data_reduit

In [ ]:
from statsmodels.tools.tools import add_constant

#function for listing vif values
def vif_values(X):
    add_constant(X)
    df=pd.Series([variance_inflation_factor(X.values, i) for i in range(X.shape[1])], index=X.columns)
    return df

In [ ]:
vif_values(df_data_reduit)

In [ ]:
Raw=drop_high_vif(X_raw,thresh=10)

In [ ]:
vif_values(Raw)

In [ ]:
Raw.insert(0,'Dates',df_data['Dates'])
df_data_reduit.insert(0,'Dates',df_data['Dates'])

In [ ]:
df_merged=Raw.merge(df_data_reduit,how='outer')
df_merged.drop_duplicates()

In [ ]:
df_merged.drop(columns='Dates',inplace=True)

In [ ]:
df_data_reduit=drop_high_vif(df_merged,thresh=10)
df_data_reduit

In [ ]:
vif_values(df_data_reduit)

In [ ]:
rf_final=RandomForestRegressor(random_state=7,n_jobs=-1)
rf_final.fit(df_data_reduit,tf.squeeze(np.asarray(y),1))

In [ ]:
# function for creating a feature importance dataframe
def feature_importance(column_names, importances):
    df = pd.DataFrame({'feature': column_names, 'feature_importance': importances}).sort_values('feature_importance', ascending = False).reset_index(drop = True)
    return df

In [ ]:
# plotting a feature importance dataframe (horizontal barchart)
def plot_feature_importance(imp_df, title):
    # figure size in inches
    plt.rcParams['figure.figsize'] = 11.7,8.27
    imp_df.columns = ['feature', 'feature_importance']
    sns.barplot(x = 'feature_importance', y = 'feature', data = imp_df, color = 'royalblue').set_title(title, fontsize = 20)

In [ ]:
imp_feat=feature_importance(df_data_reduit.columns,rf_final.feature_importances_)
plot_feature_importance(imp_feat,'feature importance')

In [ ]:
imp_feat[imp_feat['feature_importance']>0]

In [ ]:
selected = imp_feat[imp_feat['feature_importance']>0]
selected

In [ ]:
selected.feature_importance.sum()

In [ ]:
df_data_reduit_high = df_data_reduit[np.asarray(selected.feature)]
df_data_reduit_high.sort_index(axis=1,inplace=True)
df_data_reduit_high['Price'] = np.ravel(y)
df_data_reduit_high = df_data_reduit_high[df_data_reduit_high['Price']!=1]
df_data_reduit_high

In [ ]:
corr = df_data_reduit_high.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=np.bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 8))

sns.heatmap(corr,mask=mask, cmap='coolwarm',annot=True, fmt='.2f')

In [ ]:
from sklearn.ensemble import IsolationForest

anomalies=IsolationForest(contamination=0.1)

price=df_data_reduit_high['Price']
price=price.values
price=np.reshape(price,(-1,1))
anomalies.fit(price)

pred=anomalies.predict(price)

df_data_reduit_high['Anomalies'] = pred
df_data_reduit_high['Anomalies'] = df_data_reduit_high['Anomalies'].apply(lambda x: 1 if (x==-1) else 0)

# Affiche les informations sur les anomalies
print(df_data_reduit_high['Anomalies'].value_counts())

In [ ]:
import plotly.express as px

# Affiche la série

fig = px.line(x=df_data_reduit_high.index,y=df_data_reduit_high['Price'],title="Evolution du prix du BTC")
fig.add_trace(px.scatter(x=df_data_reduit_high.index,y=df_data_reduit_high['Anomalies']*df_data_reduit_high['Price'],color=df_data_reduit_high['Anomalies'].astype(np.bool)).data[0])

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

In [ ]:
#df_data_reduit_high.reset_index(drop=True,inplace=True)
df_data_reduit_high.drop(columns=['Anomalies'],inplace=True)

In [ ]:
df_data_reduit_high.insert(0,'Dates',df_data['Dates'])
df_data_reduit_high.set_index(df_data_reduit_high['Dates'])
df_data_reduit_high

In [ ]:
df_data_reduit_high.columns

In [ ]:
df_data_reduit_high.to_csv('reg_1d.csv')